# Entrenamiento de una red neuronal en MNIST con Keras

Este sencillo ejemplo demuestra cómo conectar TensorFlow Datasets (TFDS) a un modelo Keras.


Copyright 2020 The TensorFlow Datasets Authors, Licencia bajo la Licencia Apache, Versión 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/datasets/keras_example"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/datasets/keras_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/datasets/keras_example.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/datasets/keras_example.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>
</td>
</table>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

## Paso 1: Cree su tubería de entrada

Empieza por crear una tubería de entrada eficiente siguiendo los consejos de:

- La guía de [Consejos de rendimiento](https://www.tensorflow.org/datasets/performances)
- El [Mejor rendimiento con la `tf.data` API](https://www.tensorflow.org/guide/data_performance#optimize_performance)


### Carga un dataset

Cargue el dataset MNIST con los siguientes argumentos:

- `shuffle_files=True`: Los datos MNIST sólo se almacenan en un único archivo, pero para datasets más grandes con varios archivos en el disco, es una buena práctica barajarlos cuando se entrena.
- `as_supervised=True`: Devuelve una tupla `(img, label)` en lugar de un diccionario `{'image': img, 'label': label}`.

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

### Construir una tubería de entrenamiento

Aplica las siguientes transformaciones:

- `tf.data.Dataset.map`: TFDS proporciona imágenes de tipo `tf.uint8`, mientras que el modelo espera `tf.float32`. Por lo tanto, es necesario normalizar las imágenes.
- `tf.data.Dataset.cache` A medida que se ajusta el dataset en la memoria, almacenalo en caché antes de barajar para un mejor rendimiento.<br> **Nota:** Las transformaciones aleatorias deben aplicarse después del almacenamiento en caché.
- `tf.data.Dataset.shuffle`: Para una verdadera aleatoriedad, ajuste el búfer de barajado al tamaño completo del dataset. {Nota: Para datasets grandes, que no caben en la memoria, utilice `buffer_size=1000` si su sistema lo permite.
- `tf.data.Dataset.batch`: Agrupa elementos del dataset después de barajar para obtener lotes únicos en cada momento.
- `tf.data.Dataset.prefetch`: Es una buena práctica terminar la tubería precargando [por rendimiento](https://www.tensorflow.org/guide/data_performance#prefetching).

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

### Construir una tubería de evaluación

Su tubería de pruebas es similar a la de entrenamiento, con pequeñas diferencias:

- No es necesario llamar a `tf.data.Dataset.shuffle`.
- El almacenamiento en caché se realiza después de la agrupación porque los grupos pueden ser los mismos entre los tiempos.

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## Paso 2: Crear y entrenar el modelo

Conecte la tubería de entrada TFDS a un modelo Keras simple, compile el modelo y entrénelo.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)